# ECE324 

<img src = "https://geekflare.com/wp-content/uploads/2021/12/ai-projects-for-beginners.png" alt="image" />

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow
!pip install datasets
!pip install image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.models as models
from torchvision import *
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import Compose, Resize, ToTensor, ToPILImage, RandomHorizontalFlip, RandomRotation, RandomResizedCrop, ColorJitter, ToTensor

import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
import json
import urllib.request
from io import BytesIO
import pandas as pd
import uuid

import nltk
nltk.download("punkt")

from nltk import tokenize

from datasets import load_dataset
from pathlib import Path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

datasetImageList = []
datasetLabelList = []

stable_diffusion_dataset = load_dataset('poloclub/diffusiondb', 'large_first_5k')["train"]
dalle2_dataset = datasets.ImageFolder('drive/MyDrive/diffusion_data/images/')


with open('drive/MyDrive/diffusion_data/midjourney/general-01_2022_06_20.json', 'r') as f:
  data = json.load(f)
  data = data["messages"]
  print(data)
  counter = 0
  for image in data:
    if (counter > 5000):
      break
    if (len(image) > 0 and len(image[0]["attachments"]) > 0):
      req = urllib.request.Request(url=image[0]["attachments"][0]["proxy_url"], headers=headers)
      try:
        with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"images/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          datasetImageList.append(pil_image)
          datasetLabelList.append(torch.tensor(3))
          counter = counter + 1
      except Exception as e:
        print(e)
        pass

IndentationError: ignored

In [ ]:


df = pd.read_parquet('drive/MyDrive/diffusion_data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet')

for index, row in df.iterrows():
    if (index > 5000):
      break
    req = urllib.request.Request(url=row["URL"], headers=headers)
    try:
      with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"images/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          datasetImageList.append(file_path)
          datasetLabelList.append(torch.tensor(0))
    except: 
      pass


del df

In [ ]:
counter = 0

for data in dalle2_dataset:
  image = np.array(data[0])[:3]
  label = torch.tensor(2)
  datasetImageList.append(image)
  datasetLabelList.append(label)
  counter += 1

for data in stable_diffusion_dataset:
  image = np.array(data['image'])[:3]
  label = torch.tensor(1)
  datasetImageList.append(image)
  datasetLabelList.append(label)

del dalle2_dataset
del stable_diffusion_dataset

bruh
bruh
bruh
bruh


In [ ]:
def resize_and_convert(image, size=(254, 254)):
    if not isinstance(image, np.ndarray):
        image = np.array(image)
    pil_image = Image.fromarray(image).convert('RGB')
    pil_image = pil_image.resize(size)
    return np.transpose(np.array(pil_image), (2, 0, 1))

augmentations = Compose([
    RandomHorizontalFlip(p=0.5),
    RandomRotation(degrees=15),
    RandomResizedCrop(size=(254, 254), scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    ToTensor()
])


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_list, label_list, transform):
        self.image_list = image_list
        self.label_list = label_list
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = self.image_list[idx]
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        elif isinstance(image, np.ndarray):
            image = Image.fromarray(image).convert("RGB")
          
        image = image.resize((254, 254))
        label = self.label_list[idx]
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
batch_size = 64
dataset = ImageDataset(datasetImageList, datasetLabelList, augmentations)

train_size = int(len(dataset) * 0.90)
val_size = len(dataset) - train_size
train_indices = torch.randperm(len(dataset))[:train_size]
val_indices = torch.arange(train_size, len(dataset))

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Loading the previous model if it exists and then training the model based on data. Save the model after each epoch.

In [ ]:
model = torch.load("drive/MyDrive/diffusion_data/mode.pt")

num_features = model.fc.in_features
#model.fc = nn.Linear(num_features, 4)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.000001)

for param in model.parameters():
  param.requires_grad = True
num_epochs = 20
for epoch in range(num_epochs):



    # Forward pass
    model.train()
    for inputs, labels in train_dataloader:
      # Zero the gradients
      optimizer.zero_grad()

      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())

      # Compute the loss
      labels = labels.to(device)
      loss = criterion(outputs, labels)

      # Backward pass
      loss.backward()
      optimizer.step()
    print(epoch)
    torch.save(model, "drive/MyDrive/diffusion_data/mode.pt")

    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
          # Forward pass
          inputs = inputs.to(device)
          outputs = model(inputs.float())
          _, predicted = torch.max(outputs.data, 1)
          
          labels = labels.to(device)
          loss = criterion(outputs, labels)
          total_loss += loss.item()

          total_samples += labels.size(0)
          total_correct += (predicted == labels).sum().item()

    accuracy = total_correct / total_samples
    print('Validation accuracy:', accuracy)
    print("Loss", total_loss)


0
Validation accuracy: 0.9156378600823045
Loss 3.766630917787552
1
Validation accuracy: 0.8724279835390947
Loss 4.033371031284332
2
Validation accuracy: 0.8724279835390947
Loss 4.010570198297501
3
Validation accuracy: 0.8353909465020576
Loss 4.006787955760956
4
Validation accuracy: 0.8744855967078189
Loss 3.9909191727638245


KeyboardInterrupt: ignored

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

datasetImageList = []
datasetLabelList = []

stable_diffusion_dataset = load_dataset('poloclub/diffusiondb', 'large_random_1k')["train"]
dalle2_dataset = datasets.ImageFolder('drive/MyDrive/diffusion_data/images/')

counter = 0


dalle2ImageList = []
dalle2LabelList = []
for data in dalle2_dataset:
  if (counter > 500):
    break
  image = np.array(data[0])[:3]
  label = torch.tensor(0)
  dalle2ImageList.append(image)
  dalle2LabelList.append(label)
  counter += 1



stableDiffusionImageList = []
stableDiffusionLabelList = []
counter = 0
for data in stable_diffusion_dataset:
  if (counter > 500):
    break
  image = np.array(data['image'])[:3]
  label = torch.tensor(0)
  stableDiffusionImageList.append(image)
  stableDiffusionLabelList.append(label)
  counter += 1

del dalle2_dataset
del stable_diffusion_dataset



midjourneyImageList = []
midjourneyLabelList = []
with open('/content/drive/MyDrive/diffusion_data/midjourney/general-01_2022_07_04.json', 'r') as f:
  data = json.load(f)
  data = data["messages"]
  counter = 0
  for image in data:
    if (counter > 500):
      break
    if (len(image) > 0 and len(image[0]["attachments"]) > 0):
      req = urllib.request.Request(url=image[0]["attachments"][0]["proxy_url"], headers=headers)
      try:
        with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"mid/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          midjourneyImageList.append(pil_image)
          midjourneyLabelList.append(torch.tensor(1))
          counter = counter + 1
      except Exception as e:
        print(e)
        pass


realImageList = []
realLabelList = []
df = pd.read_parquet('/content/drive/MyDrive/diffusion_data/part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet')

for index, row in df.iterrows():
    if (index > 500):
      break
    req = urllib.request.Request(url=row["URL"], headers=headers)
    try:
      with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"real/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          realImageList.append(file_path)
          realLabelList.append(torch.tensor(0))
    except: 
      pass


del df

Generating train split: 0 examples [00:00, ? examples/s]

Dataset diffusiondb downloaded and prepared to /root/.cache/huggingface/datasets/poloclub___diffusiondb/large_random_1k/0.9.1/547894e3a57aa647ead68c9faf148324098f47f2bc1ab6705d670721de9d89d1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'mid/0987222fcfef47d28a2188c6f5645fa9.jpg'
[Errno 2] No such file or directory: 'mid/e3c92afca3274bf480c15af6a374f69a.jpg'
[Errno 2] No such file or directory: 'mid/1f1f85909b5f4a5dacd9a74fe0672add.jpg'
[Errno 2] No such file or directory: 'mid/2b8b4b540f8941d7ae8533949228040f.jpg'
[Errno 2] No such file or directory: 'mid/651baa14645c4f47bcedd05ff599cd42.jpg'
[Errno 2] No such file or directory: 'mid/8495fe912dd74508bc86c82adeaa62a5.jpg'
[Errno 2] No such file or directory: 'mid/d01e4acc683a433496a852a90dfaaf19.jpg'
[Errno 2] No such file or directory: 'mid/e9481b852a504b10be0e997681bd9006.jpg'
[Errno 2] No such file or directory: 'mid/a3cf3834b79a442fbd73a02ce6cd3681.jpg'
[Errno 2] No such file or directory: 'mid/90bd974070eb4324bf3ffc0bde33304c.jpg'
[Errno 2] No such file or directory: 'mid/42e63427b2bd4018aa766da4a67a6243.jpg'
[Errno 2] No such file or directory: 'mid/1bfc9369888946d9a5f6cfbf671988c4.jpg'
[Errno 2] No such file or directory: 'mi

In [ ]:
augmentations = Compose([
    ToTensor()
])
batch_size = 64
realDataset = ImageDataset(realImageList, realLabelList, transform=augmentations)
midjourneyDataset = ImageDataset(midjourneyImageList, midjourneyLabelList, transform=augmentations)
stableDiffusionDataset = ImageDataset(stableDiffusionImageList, stableDiffusionLabelList, transform=augmentations)
dalle2Dataset = ImageDataset(dalle2ImageList, dalle2LabelList, transform=augmentations)


realDataloader = DataLoader(realDataset, batch_size=batch_size)
midjourneyDataloader = DataLoader(midjourneyDataset, batch_size=batch_size)
stableDiffusionDataloader = DataLoader(stableDiffusionDataset, batch_size=batch_size)
dalle2Dataloader = DataLoader(dalle2Dataset, batch_size=batch_size)

In [ ]:
model = torch.load("drive/MyDrive/diffusion_data/new_model50.pt")


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()

model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in realDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Real:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")

model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in midjourneyDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Midjourney:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")
model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in stableDiffusionDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      print(predicted)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Stable Diffusion:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")
model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in dalle2Dataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Dalle 2:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")






Real:
Validation accuracy: 0.97
Loss 1.7885061204433441

Midjourney:
Validation accuracy: 0.812375249500998
Loss 3.646006256341934

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.models import LdaModel

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

datasetImageList = []
datasetLabelList = []
datasetPromptList = []

with open('drive/MyDrive/diffusion_data/midjourney/general-01_2022_06_20.json', 'r') as f:
  data = json.load(f)
  data = data["messages"]
  print(data)
  counter = 0
  for image in data:
    if (counter % 100 == 0):
      print(counter)
    if (counter > 1500):
      break
    if (len(image) > 0 and len(image[0]["attachments"]) > 0):
      req = urllib.request.Request(url=image[0]["attachments"][0]["proxy_url"], headers=headers)
      try:
        with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"images/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          datasetImageList.append(pil_image)
          datasetLabelList.append(torch.tensor(1))
          datasetPromptList.append((image[0]["content"]))
          counter = counter + 1
      except Exception as e:
        print(e)
        pass




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100
200
300
400
500
600
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [ ]:


df = pd.read_parquet('drive/MyDrive/diffusion_data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet', columns=["URL", "TEXT"])

real_image_list = []
real_label_list = []
real_prompt_list = []

for index, row in df.iterrows():
    #print(row)
    if (len(real_image_list) % 100 == 0):
      print(len(real_image_list))
    if (len(real_image_list) > 1500):
      break
    req = urllib.request.Request(url=row["URL"], headers=headers)
    try:
      with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"images/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          real_image_list.append(file_path)
          real_label_list.append(torch.tensor(0))
          real_prompt_list.append((row["TEXT"]))
    except: 
      pass


del df

0
0
100
100
200
200
300
400
500
600
600
700
800
900
1000
1100
1200
1300
1300
1300
1400
1500
1600
1700
1800
1900
2000


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter

# Array of strings (prompts)
prompts = datasetPromptList + real_prompt_list

print(prompts)
# Convert the text data into numerical data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(prompts)

# Define the number of clusters you want (change this to fit your needs)
num_clusters = 100

# Apply K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(X)

# Group prompts by their cluster assignment
clusters = {i: [] for i in range(num_clusters)}
for i, prompt in enumerate(prompts):
    clusters[kmeans.labels_[i]].append(prompt)

# Get the count of prompts in each cluster and sort them in descending order
cluster_counts = [(cluster_id, len(prompts)) for cluster_id, prompts in clusters.items()]
sorted_clusters = sorted(cluster_counts, key=lambda x: x[1], reverse=True)

# Take the top 50 most popular clusters
top_clusters = sorted_clusters[:50]


midjourney_image_list = []
midjourney_label_list = []

final_real_image_list = []
final_real_label_list = []

print("Top 50 most popular clusters:")
for cluster_id, count in top_clusters:
    #print(f"Cluster {cluster_id} (count: {count}): {', '.join(clusters[cluster_id])}")
    for text in clusters[cluster_id]:
      if text in datasetPromptList:
        index = datasetPromptList.index(text)
        midjourney_image_list.append(datasetImageList[index])
        midjourney_label_list.append(datasetLabelList[index])
      elif text in real_prompt_list:
        index = real_prompt_list.index(text)
        final_real_image_list.append(real_image_list[index])
        final_real_label_list.append(real_label_list[index])


print(len(midjourney_image_list))
print(len(final_real_image_list))

del prompts
del datasetPromptList
del real_prompt_list



['**Bill Sienkiewicz technology | Jack Kirby machines |  retro futuristic  ghibli matte painting --aspect 9:19 --no faces** - Upscaled (Max) by <@986404200177094667> (relaxed)', '**<https://s.mj.run/vN6ggUx7POc> --ar 9:16** - Upscaled by <@713319244347211797> (metered, fast)', '**Bill Sienkiewicz technology | Jack Kirby machines |  retro futuristic  ghibli matte painting --aspect 9:19 --no faces** - Upscaled (Max) by <@986404200177094667> (relaxed)', '**Bill Sienkiewicz technology | Jack Kirby machines |  retro futuristic  ghibli matte painting --aspect 9:19 --no faces** - Upscaled (Max) by <@986404200177094667> (relaxed)', '**<https://s.mj.run/QIgF9-KsZaU> cats with many eyes floating in colorful glowing swirling whisps, occult inspired, emerging from the void, shallow depth of field --aspect 7:2** - <@412798156477169664> (relaxed)', '**bent twisted impossible shape mysterious foreboding Tim burton tree --aspect 9:19** - Upscaled (Max) by <@986404200177094667> (relaxed)', '**<https://

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Top 50 most popular clusters:
1417
1941


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(kmeans.cluster_centers_, kmeans2.cluster_centers_)

# Define similarity threshold
similarity_threshold = 0.9

# Cross-match clusters from both datasets
matched_clusters = []
for i in range(100):
    best_match = similarity_scores[i].argmax()
    if similarity_scores[i, best_match] >= similarity_threshold:
        matched_clusters.append((i, best_match))

# Extract cross-matched clusters
X1_matched = X1[kmeans.labels_ == matched_clusters[0][0]]
X2_matched = X2[kmeans2.labels_ == matched_clusters[0][1]]
for i in range(1, len(matched_clusters)):
    X1_matched = np.concatenate((X1_matched, X1[kmeans.labels_ == matched_clusters[i][0]]), axis=0)
    X2_matched = np.concatenate((X2_matched, X2[kmeans2.labels_ == matched_clusters[i][1]]), axis=0)

In [ ]:
augmentations = Compose([
    ToTensor()
])

batch_size = 64
dataset_image_list = final_real_image_list + midjourney_image_list
dataset_label_list = final_real_label_list + midjourney_label_list

del final_real_image_list
del midjourney_image_list
del final_real_label_list
del midjourney_label_list
print(len(dataset_image_list))
print(len(dataset_label_list))
dataset = ImageDataset(dataset_image_list, dataset_label_list, augmentations)

train_size = int(len(dataset) * 0.90)
val_size = len(dataset) - train_size
train_indices = torch.randperm(len(dataset))[:train_size]
val_indices = torch.arange(train_size, len(dataset))

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


3358
3358


In [ ]:
model = torch.load("drive/MyDrive/diffusion_data/new_model50.pt")


#model = models.resnet50(pretrained = True)
#num_features = model.fc.in_features
#model.fc = nn.Linear(num_features, 2)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

for param in model.parameters():
  param.requires_grad = True
num_epochs = 20
for epoch in range(num_epochs):



    # Forward pass
    model.train()
    for inputs, labels in train_dataloader:
      # Zero the gradients
      optimizer.zero_grad()

      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())

      # Compute the loss
      labels = labels.to(device)
      loss = criterion(outputs, labels)

      # Backward pass
      loss.backward()
      optimizer.step()
    print(epoch)
    torch.save(model, "drive/MyDrive/diffusion_data/new_model50.pt")

    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
          # Forward pass
          inputs = inputs.to(device)
          outputs = model(inputs.float())
          _, predicted = torch.max(outputs.data, 1)
          
          labels = labels.to(device)
          loss = criterion(outputs, labels)
          total_loss += loss.item()

          total_samples += labels.size(0)
          total_correct += (predicted == labels).sum().item()

    accuracy = total_correct / total_samples
    print('Validation accuracy:', accuracy)
    print("Loss", total_loss)


0
Validation accuracy: 0.9434523809523809
Loss 1.9952584207057953
1
Validation accuracy: 0.9136904761904762
Loss 2.050030678510666
2
Validation accuracy: 0.9523809523809523
Loss 1.8687182366847992


KeyboardInterrupt: ignored

In [ ]:
augmentations = Compose([
    ToTensor()
])
batch_size = 64
realDataset = ImageDataset(realImageList, realLabelList, transform=augmentations)
midjourneyDataset = ImageDataset(midjourneyImageList, midjourneyLabelList, transform=augmentations)
stableDiffusionDataset = ImageDataset(stableDiffusionImageList, stableDiffusionLabelList, transform=augmentations)
dalle2Dataset = ImageDataset(dalle2ImageList, dalle2LabelList, transform=augmentations)


realDataloader = DataLoader(realDataset, batch_size=batch_size)
midjourneyDataloader = DataLoader(midjourneyDataset, batch_size=batch_size)
stableDiffusionDataloader = DataLoader(stableDiffusionDataset, batch_size=batch_size)
dalle2Dataloader = DataLoader(dalle2Dataset, batch_size=batch_size)

NameError: ignored

In [ ]:
model = torch.load("drive/MyDrive/diffusion_data/new_model.pt")


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()

model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in realDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Real:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")

model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in midjourneyDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Midjourney:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")
model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in stableDiffusionDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      print(predicted)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Stable Diffusion:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")
model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in dalle2Dataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Dalle 2:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")




